In [2]:
# вариант очистки данных максимально приближенный к оригиналу 
import re 
import os
import json

# очистка данных

important_symbols = ['№', '-', ')', '(', '.', ',', ':',';', '«', '»', ' ']
latin_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'

def cleanser(text):
 
    words = text.split()
    final = []
    final.append('\n')
    temp = ''

    for word in words: 
        for w in word:
            if w not in latin_letters:
                if w in important_symbols:
                    temp += w
                elif w.isalnum():
                    temp += w   
        final.append(temp)
        temp = ''
                        
    final = " ".join(final) # склеить список из строк
    final = re.sub(r' {1,}', ' ', final) # удалить много пробелов
    
    return (final)    


def clean_docs(docs):
    
    final_text = []
    for i, doc in enumerate(docs):
        # if i == 0 or i == 1:    # не трогать название документа
        #     final_text.append(doc)
        # else:
            clean_doc = cleanser(doc)
            final_text.append(clean_doc)

    return (final_text)

In [1]:
# очистка данных перед jsocn
from nltk.corpus import stopwords
import string
import re 
import os
import json


# очистка данных
months = ["Январ", "Феврал", "Март", "Апрел", "Ма", "Июн", "Июл", "Август", "Сентябр", "Октябр", "Ноябр", "Декабр"]
pattern = r'\b(?:' + '|'.join([month + '[а-я]*' for month in months]) + r')\b[.,]?' 
laws_links = []

def remove_stops(text, stops):

    text = re.sub(r'[а-яА-Я]\)\s', ' ', text) # удалить а), б) ...
    text = re.sub(r'\b[IVXLCDMivxlcdm]+\b', '', text) # удалить все римские цифры
    # text = re.sub(pattern, '', text, flags=re.IGNORECASE) # удалить месяцы
    
    #text = re.sub(r'\bг\.', '', text)
    #text = re.sub(r'\s\d{1,2}\.\s|\t\d{1,2}\.\s', ' ', text) # удалить 1. 2. ...
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'№\s', '№', text)# конкатенация №ФЗ123
    text = re.sub(r'(\t\d\s)|(\s\d\s)|(\s\d\t)', ' ', text)
    text = re.sub('•|№', '', text) # удаление мусора по типу '•'

    words = text.split()
    final = []
    final.append(' ')
    for word in words: # удалить стоп-слова
        if word not in stops:
            final.append(word)
    final = " ".join(final) # склеить список из строк
    final = final.translate(str.maketrans("", "", string.punctuation)) # удалить пунктуацию
    final = re.sub(r'(т\sд)|(\sт\sд)|(\sт\s)|((\sг\s))', '', final)
    
    #закинем в конец дока ссылки на законы в документе (по другому никак если оставлять только их)
    # z = text.split(' ')
    # for ind, i in enumerate(z):
    #     if '№' in i:
    #         laws_links.append(i)
            
    
    # final = "".join([i for i in final if not i.isdigit()]) # удалить все цифры
    final = re.sub(r'\t', ' ', final)
    final = re.sub(r'\s{1,}', ' ', final) # удалить много пробелов
    
    return (final)    


def clean_docs(docs):
    stops = stopwords.words('russian')
    final_text = []
    for i, doc in enumerate(docs):
        # if i == 0 or i == 1:    # не трогать название документа
        #     final_text.append(doc)
        # else:
            clean_doc = remove_stops(doc, stops)
            final_text.append(clean_doc)
            
    # if laws_links:
    #     final_text.append('\n')         
    #     final_text.append(' '.join(laws_links))
    
    # laws_links.clear
        
    return (final_text)

In [13]:
r_path = '/Users/gleb/Desktop/InternetLab/internetlab/docs/row_docs/row_region/'
c_path = '/Users/gleb/Desktop/InternetLab/internetlab/docs/clean_docs/reg_clean/'

raw_doc_list = os.listdir(r_path)
clean_doc_list = os.listdir(c_path)
for doc in raw_doc_list:
    if '.txt' in doc:
        file  = open(os.path.join(r_path, doc), 'r', encoding='utf-8')
        cleaned_docs = clean_docs(file)
        # запись в файл
    
        document = os.path.join(c_path, 'c_' + doc)
        with open(document, "w", encoding="utf-8") as d:
            for i in cleaned_docs:
                d.write(i)
        

FileNotFoundError: [Errno 2] No such file or directory: '/Users/gleb/Desktop/InternetLab/internetlab/docs/row_docs/row_region/'

2021 год региональные



In [4]:
# очистка доков для tf-idf

r_path = '/Users/gleb/Desktop/internerLAB/internetlab/docs/clean_docs/reg_clean/reg_over_2021-2023/2023/raw'
c_path = '/Users/gleb/Desktop/internerLAB/internetlab/docs/clean_docs/reg_clean/reg_over_2021-2023/2023/clean'

raw_doc_list = os.listdir(r_path)
clean_doc_list = os.listdir(c_path)
for doc in raw_doc_list:
    if '.txt' in doc:
        file  = open(os.path.join(r_path, doc), 'r', encoding='utf-8')
        cleaned_docs = clean_docs(file)
        # запись в файл
    
        document = os.path.join(c_path, doc)
        with open(document, "w", encoding="utf-8") as d:
            for i in cleaned_docs:
                d.write(i)
        

In [5]:
# запись в json док-ов по годам
folder_path = '/Users/gleb/Desktop/internerLAB/internetlab/docs/clean_docs/reg_clean/reg_over_2021-2023/2023/clean'
json_path = '/Users/gleb/Desktop/internerLAB/internetlab/docs/clean_docs/reg_clean/reg_over_2021-2023/2023/2023.json'

titles = []
texts = []

for filename in os.listdir(folder_path):
    if filename.endswith(".txt") and 'unrec' not in filename:
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
            lines = file.readlines()
            titles.append(filename)  # Добавляем название документа 
            texts.extend(lines)  # Добавляем текст 
            
data = {
    "title": titles,
    "text": texts
}

# Записываем данные в JSON файл
with open(json_path, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)